In [1]:
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import sqlite3

In [2]:
def ricomponi_mappa(cartella):
    # Definisci le dimensioni di ciascuna immagine e la griglia totale
    larghezza_img = 256
    altezza_img = 256
    righe = 32
    colonne = 128
    
    # Dimensioni dell'immagine finale
    larghezza_totale = larghezza_img * colonne
    altezza_totale = altezza_img * righe
    
    # Crea una nuova immagine vuota con le dimensioni totali
    immagine_finale = Image.new('RGB', (larghezza_totale, altezza_totale))
    
    # Loop attraverso tutte le posizioni per inserire le immagini
    for riga in range(16, 48):
        for colonna in range(0, 128):
            # Costruisci il nome del file
            nome_file = f"image_{riga}_{colonna}.png"
            percorso_file = os.path.join(cartella, nome_file)
            
            # Carica l'immagine
            if os.path.exists(percorso_file):
                img = Image.open(percorso_file)
                # Calcola la posizione in cui incollare l'immagine
                posizione_x = (colonna) * larghezza_img
                posizione_y = (riga - 16) * altezza_img
                
                # Incolla l'immagine nel punto corretto
                immagine_finale.paste(img, (posizione_x, posizione_y))
            else:
                print(f"File {percorso_file} non trovato.")
    
    return immagine_finale

In [5]:
def immagine_a_matrice(immagine):
    # Converti l'immagine in formato RGB (se non lo è già)
    immagine = immagine.convert('RGB')
    
    # Ottieni i dati dell'immagine come array NumPy
    array_dati = np.array(immagine)
    
    # Ottieni le dimensioni dell'immagine
    altezza, larghezza, _ = array_dati.shape
    
    # Inizializza la matrice di output
    matrice_pixel = []

    # Utilizza tqdm per mostrare una barra di progresso
    for i in tqdm(range(altezza), desc="Conversione dell'immagine in matrice"):
        riga = []
        for j in range(larghezza):
            riga.append(tuple(array_dati[i, j]))
        matrice_pixel.append(riga)
    
    return matrice_pixel

def estrai_sotto_matrice(matrice, n, m, l):
    altezza, larghezza, _ = matrice.shape
    
    # Calcola i limiti della sotto-matrice
    half_l = l // 2
    top = max(0, n - half_l)
    bottom = min(altezza, n + half_l + 1)
    left = max(0, m - half_l)
    right = min(larghezza, m + half_l + 1)
    
    # Creiamo una matrice vuota l x l con valori di default (0, 0, 0)
    sotto_matrice = np.zeros((l, l, 3), dtype=matrice.dtype)
    
    # Calcola gli offset per gestire i bordi
    offset_top = top - (n - half_l)
    offset_left = left - (m - half_l)
    
    # Copia i valori dalla matrice principale alla sotto-matrice
    sotto_matrice[offset_top:offset_top + (bottom - top), offset_left:offset_left + (right - left)] = matrice[top:bottom, left:right]
    
    return sotto_matrice

In [ ]:
def salva_matrice_in_db(matrice, nome_db):
    conn = sqlite3.connect(nome_db)
    cur = conn.cursor()
    
    # Crea la tabella per i pixel se non esiste già
    cur.execute('''
        CREATE TABLE IF NOT EXISTS pixels (
            x INTEGER,
            y INTEGER,
            r INTEGER,
            g INTEGER,
            b INTEGER
        )
    ''')
    
    # Inserisce i dati della matrice nel database
    altezza = len(matrice)
    larghezza = len(matrice[0])
    
    for i in tqdm(range(altezza), desc="Salvataggio della matrice nel DB"):
        for j in range(larghezza):
            r, g, b = matrice[i][j]
            cur.execute('INSERT INTO pixels (x, y, r, g, b) VALUES (?, ?, ?, ?, ?)', (i, j, r, g, b))
    
    # Commit e chiusura della connessione
    conn.commit()
    conn.close()

In [6]:
immagine_finale = ricomponi_mappa("tante_immagini")


In [3]:
matrice_finale = immagine_a_matrice(immagine_finale)

NameError: name 'immagine_a_matrice' is not defined

In [ ]:
nome_db = 'matrice_pixels.db'
salva_matrice_in_db(matrice, nome_db)

In [ ]:
#sotto_matrice = estrai_sotto_matrice(matrice, n, m, l)
#print(sotto_matrice)